# Initiation au Machine Learning

In [19]:
import pandas as pd

## Importation des données

In [20]:
# Vecteur qui contiendra toutes les entrées issues de Wikipédia
X = []
# Vecteur qui contiendra la langue de sortie des entrées correspondantes
Y = []

# On va importer les données (contenues dans des fichiers textes)
with open("data1.txt", encoding="utf-8") as f1, open("data2.txt", encoding="utf-8") as f2, open("data3.txt", encoding="utf-8") as f3:
    for line in f1:
        y = line[0]
        x = line[2:]
        X.append(x)
        Y.append(y)
    for line in f2:
        y = line[0]
        x = line[2:]
        X.append(x)
        Y.append(y)
    for line in f3:
        y = line[0]
        x = line[2:]
        X.append(x)
        Y.append(y)
        
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

## Vectoriser les entrées
Le but est de passer d'un texte d'une longueur arbitraire à un vecteur à n_dimensions N_features

In [21]:
# On importe une fonction qui convertit des contenus de documents texte en une matrice de comptage de mots
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
# On détermine qu'un token (fragment de mot) fais la taille d'un mot (à partir de 2 caractères)
vectorizer = CountVectorizer(
    input="content",
    analyzer="word",
    strip_accents=None,
    stop_words=None,
    binary=True,
    ngram_range=(1,2)
)

In [23]:
# On tokenize (on "distingue" les mots de la phrase)
tokenizer = vectorizer.build_tokenizer()
tokenizer("salut le monde, ça va ?")

['salut', 'le', 'monde', 'ça', 'va']

In [24]:
# On analyse (on distingue des suites de n mots)
analyzer = vectorizer.build_analyzer()
analyzer("salut le monde, ça va ?")

['salut',
 'le',
 'monde',
 'ça',
 'va',
 'salut le',
 'le monde',
 'monde ça',
 'ça va']

In [25]:
# On importe une fonction qui segmente les données en jeux d'entrainement et de tests)
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    train_size=0.7
)

C:\Users\Xepyon\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [27]:
# On jette un oeil au format des données d'entrainement de X
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9040 entries, 8770 to 10720
Data columns (total 1 columns):
0    9040 non-null object
dtypes: object(1)
memory usage: 141.2+ KB


In [28]:
# Même chose pour les données de test
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3875 entries, 7316 to 10154
Data columns (total 1 columns):
0    3875 non-null object
dtypes: object(1)
memory usage: 60.5+ KB


In [29]:
# On importe une fonction qui applique des transformations aux données afin d'obtenir une évaluation finale
from sklearn.pipeline import Pipeline
# On importe un classifieur pour les modèles Multinomiaux
from sklearn.naive_bayes import MultinomialNB

In [30]:
# On crée un pipeline de traitement, puis on vectorise les données textuelles et enfin on les classifie
classifier = MultinomialNB()
pipeline = Pipeline(
    [
        ("vectorizer", vectorizer),
        ("classifier", classifier)
    ]
)

In [31]:
# On entraine le classifier sur les données d'entrainement
pipeline.fit(X_train[0], Y_train)
print("fit done")

C:\Users\Xepyon\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


fit done


In [32]:
# On utilise le pipeline entrainé sur les données de test
Y_predicted = pipeline.predict(X_test[0])

In [33]:
# On regarde les résultats des 10 premières lignes
Y_predicted[1:10]

array(['1', '0', '1', '0', '0', '0', '1', '0', '1'], dtype='<U1')

In [34]:
# On importe une fonction qui calcule le f1_score (le rapport entre la précision et le rappel)
from sklearn.metrics import f1_score
# On calcule le f1_score pour toutes les prédictions effectuées
score = f1_score(
    Y_test[0],
    Y_predicted,
    average="micro"
)
print("score f1", score)

score f1 0.9953548387096774


On obtient un score f1 de 0.9953 sur les données d'entrainement (soit 99.53% de réussite à la classification), on est donc plutôt pas mal

In [35]:
# Maintenant on va tester la même chose sur les données qui sont réellement à évaluer (les données de test)
Y_predicted = pipeline.predict(X_train[0])
# Calcul du score f1
score = f1_score(
    Y_train[0],
    Y_predicted,
    average="micro"
)
print("score f1", score)

score f1 0.9969026548672566


On obtient un score f1 de 0.9969 sur les données de test (soit 99.69% de réussite à la classification), c'est un peu mieux que sur les données d'entrainement, donc à priori on est bien